# Query & lookup data

Query data using [SQLAlchemy](https://github.com/sqlalchemy/sqlalchemy) `select` statements.

Use lookups to auto-complete categorical query conditions.

In [ ]:
# initialize a test instance for this notebook
!lamin init --storage ./myobjects

In [ ]:
import lamindb as ln

In [ ]:
ln.track()
ln.add(ln.File(ln.dev.datasets.df_iris(), name="iris"))
ln.add(ln.File(ln.dev.datasets.file_fastq()));

## Basic select statements

In [ ]:
user_handles = ln.User.lookup(field="handle")

With auto-complete, we find a user:

In [ ]:
user_handles.testuser1

You can readily use it on one of the fields:

In [ ]:
user = ln.select(ln.User, handle=user_handles.testuser1).one()

In [ ]:
user

Query all files created by that user:

In [ ]:
ln.select(ln.File).where(ln.File.created_by == user).df()

To access the query results encoded in `stmt` (a {class}`~lamindb.dev.db.SelectStmt`), execute it with one of

- `.all()`: A list of records.
- `.df()`: A dataframe with each record stored as a row.
- `.one()`: Exactly one record. Will raise an error if there is none.
- `.one_or_none()`: Either one record or `None` if there is no query result.

For example:

In [ ]:
ln.select(ln.File).where(ln.File.created_by == user).all()[:3]

## Joined views

Say we want all user information in this table.

In [ ]:
ln.select(ln.User, ln.File).join(ln.File.created_by).df().head()

Say, we only want the user handle.

In [ ]:
ln.select(ln.User.handle, ln.File).join(ln.File.created_by).df().head()

Say, we only want selected information from all tables.

In [ ]:
ln.select(ln.File.name, ln.File.suffix, ln.File.size, ln.User.handle).join(
    ln.File.created_by
).df().head()

## More filtering

Let us subset to just the parquet files - we know it's exactly a single one. So we can get the record using `.one()`.

In [ ]:
ln.select(ln.File, suffix=".parquet", created_by_id=user.id).df()

Or subset to files greater than 10kB. Here, we can't use keyword arguments, but need an explicit where statement.

In [ ]:
stmt = (
    ln.select(ln.File)
    .where(ln.File.size > 1e3)
    .join(ln.File.created_by)
    .where(ln.User.handle == user_handles.testuser1)
)
stmt.df()

Or select a notebook based on a substring in the name:

In [ ]:
ln.select(ln.Transform, type="notebook").where(ln.Transform.title.contains("data")).df()

## Reference

### and

In [ ]:
ln.select(ln.File, name="iris", suffix=".parquet").first()

In [ ]:
ln.select(ln.File).where(ln.File.name == "iris", ln.File.suffix == ".parquet").first()

### or

In [ ]:
ln.select(ln.File).where(
    (ln.File.suffix == ".parquet") | (ln.File.suffix == ".fastq.gz")
).df()

### in

In [ ]:
ln.select(ln.File).where(ln.File.suffix.in_([".parquet", ".fastq.gz"])).df()

### order by

In [ ]:
ln.select(ln.Run).order_by(ln.Run.created_at.desc()).df()

### contains

In [ ]:
ln.select(ln.Transform).where(ln.Transform.title.contains("query")).df()

### startswith

In [ ]:
ln.select(ln.Transform).where(ln.Transform.title.startswith("Query")).df()

In [ ]:
!lamin delete myobjects